
This code is based on: https://github.com/mdeff/cnn_graph and https://nbviewer.jupyter.org/github/mdeff/cnn_graph/blob/outputs/usage.ipynb

Reproducing license notice from https://github.com/mdeff/cnn_graph MIT License

Copyright (c) 2016 Michaël Defferrard

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


In [0]:
!pip install tensorflow-gpu=="1.14"
!pip install numpy matplotlib
!pip install keras

In [0]:
# Downloading the repo
!git clone https://github.com/Matimath/cnn_graph

Cloning into 'cnn_graph'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 602 (delta 11), reused 22 (delta 8), pack-reused 576
Receiving objects: 100% (602/602), 438.42 KiB | 4.21 MiB/s, done.
Resolving deltas: 100% (382/382), done.


In [0]:
# Importing necessary modules from cnn_graph
%load_ext autoreload
from cnn_graph.lib import models, graph, coarsening, utils
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


hello world


In [0]:
# Load dataset
import keras
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Using TensorFlow backend.


4423680/4422102 [==============================] - 1s 0us/step


In [0]:
# Create an underlying graph as 28x28 nearest neighbours
z = graph.grid(28)
dist, idx = graph.distance_sklearn_metrics(z, k=8, metric='euclidean')
A = graph.adjacency(dist, idx).astype(np.float32)
print('d = |V| = {}, k|V| < |E| = {}'.format(0, A.nnz))


d = |V| = 0, k|V| < |E| = 6396


In [0]:
# Given grid_size it returns laplacian of 8-NN grid_size x grid_size.
def get_laplacian_of_size(grid_size):
    z = graph.grid(grid_size)
    dist, idx = graph.distance_sklearn_metrics(z, k=8, metric='euclidean')
    A = graph.adjacency(dist, idx).astype(np.float32)
    return graph.laplacian(A, normalized=True)

In [0]:
# Reshaping dataset
X_train = train_images.reshape((60000, 28*28)) 
X_test = test_images.reshape((10000, 28*28))

Layer 0: M_0 = |V| = 912 nodes (128 added),|E| = 3198 edges
Layer 1: M_1 = |V| = 456 nodes (43 added),|E| = 1456 edges
Layer 2: M_2 = |V| = 228 nodes (9 added),|E| = 689 edges
Layer 3: M_3 = |V| = 114 nodes (0 added),|E| = 328 edges


In [0]:
# Create laplacians for pooled graphs
L = [get_laplacian_of_size(s) for s in [28,14,7]]

# 3 Graph ConvNet

Here we apply the graph convolutional neural network to signals lying on graphs. After designing the architecture and setting the hyper-parameters, the model takes as inputs the data matrix $X$, the target $y$ and a list of graph Laplacians $L$, one per coarsening level.

The data, architecture and hyper-parameters are absolutely *not engineered to showcase performance*. Its sole purpose is to illustrate usage and functionality.

In [0]:
params = dict()
params['dir_name']       = 'demo'
params['num_epochs']     = 120
params['batch_size']     = 100
params['eval_frequency'] = 200

# Building blocks.
params['filter']         = 'chebyshev5'
params['brelu']          = 'b1relu'
params['pool']           = 'apool1'

# Number of classes.
C = train_labels.max() + 1
#assert C == np.unique(y).size
n_train = 60000
# Architecture.
params['F']              = [16,64,64,256]  # Number of graph convolutional filters.
params['K']              = [3,3,3,3]  # Polynomial orders.
params['p']              = [2,1, 2, 1]    # Pooling sizes.
params['M']              = [512, C]  # Output dimensionality of fully connected layers.

# Optimization.
params['regularization'] = 5e-4
params['dropout']        = 1
params['learning_rate']  = 1e-3
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['decay_steps']    = n_train / params['batch_size']

In [0]:
model = models.cgcnn(L, **params)
accuracy, loss, t_step = model.fit(X_train, train_labels, X_test, test_labels)


NN architecture
  input: M_0 = 784
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 784 * 16 / 2 = 6272
    weights: F_0 * F_1 * K_1 = 1 * 16 * 3 = 48
    biases: F_1 = 16
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 196 * 64 / 1 = 12544
    weights: F_1 * F_2 * K_2 = 16 * 64 * 3 = 3072
    biases: F_2 = 64
  layer 3: cgconv3
    representation: M_2 * F_3 / p_3 = 196 * 64 / 2 = 6272
    weights: F_2 * F_3 * K_3 = 64 * 64 * 3 = 12288
    biases: F_3 = 64
  layer 4: cgconv4
    representation: M_3 * F_4 / p_4 = 49 * 256 / 1 = 12544
    weights: F_3 * F_4 * K_4 = 64 * 256 * 3 = 49152
    biases: F_4 = 256
  layer 5: fc1
    representation: M_5 = 512
    weights: M_4 * M_5 = 12544 * 512 = 6422528
    biases: M_5 = 512
  layer 6: logits (softmax)
    representation: M_6 = 10
    weights: M_5 * M_6 = 512 * 10 = 5120
    biases: M_6 = 10







hello world
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Ins

In [0]:
rotated_test1 = np.reshape(np.rot90(np.reshape(X_test, (10000,28,28)),1, (1,2)), (10000,28*28))
rotated_test2 = np.reshape(np.rot90(np.reshape(X_test, (10000,28,28)),2, (1,2)), (10000,28*28))
rotated_test3 = np.reshape(np.rot90(np.reshape(X_test, (10000,28,28)),3, (1,2)), (10000,28*28))


In [0]:
res = model.evaluate(rotated_test1, test_labels)
print(res[0])
res = model.evaluate(rotated_test2, test_labels)
print(res[0])
res = model.evaluate(rotated_test3, test_labels)
print(res[0])

INFO:tensorflow:Restoring parameters from /content/cnn_graph/lib/../checkpoints/demo/model-72000
accuracy: 86.66 (8666 / 10000), f1 (weighted): 86.65, loss: 6.31e-01
time: 1s (wall 1s)
INFO:tensorflow:Restoring parameters from /content/cnn_graph/lib/../checkpoints/demo/model-72000
accuracy: 86.38 (8638 / 10000), f1 (weighted): 86.35, loss: 6.36e-01
time: 1s (wall 1s)
INFO:tensorflow:Restoring parameters from /content/cnn_graph/lib/../checkpoints/demo/model-72000
accuracy: 86.58 (8658 / 10000), f1 (weighted): 86.52, loss: 6.34e-01
time: 1s (wall 1s)


In [0]:
res = model.evaluate(X_test, test_labels)
print(res[0])

INFO:tensorflow:Restoring parameters from /content/cnn_graph/lib/../checkpoints/demo/model-72000
accuracy: 86.87 (8687 / 10000), f1 (weighted): 86.82, loss: 6.22e-01
time: 1s (wall 1s)
